In [1]:
import json
import os

object_name = "driller"
results_path = os.path.join("results", object_name)
print(f"Results path: {results_path}")
# list all json files in results_path
jsonl_files = [f for f in os.listdir(results_path) if f.endswith('.jsonl')]
jsonl_files

Results path: results\driller


['results_driller_analysis_batch_1.jsonl',
 'results_driller_analysis_batch_2.jsonl',
 'results_driller_analysis_batch_3.jsonl',
 'results_driller_analysis_batch_4.jsonl']

In [2]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

def split_custom_id(custom_id):
    custom_id = ''.join(custom_id.split('_')[2:])
    custom_id = '/'.join(custom_id.split('-')[1:])
    custom_id = custom_id.split('.')[0] + '.png'
    return custom_id

def process_line(line):
    custom_id = line['custom_id']
    response = line['response']
    response = json.loads(response['body']['choices'][0]['message']['content'])
    components = ', '.join(response['component'])
    order = custom_id.split('_')[1]
    file_path = split_custom_id(custom_id)
    return {
        "order": order,
        "custom_id": custom_id,
        "file_path": file_path,
        "components": components
    }

In [4]:
import pandas as pd

rows = []
for jsonl_file in jsonl_files:
    file_path = os.path.join(results_path, jsonl_file)
    print(f"Loading {file_path}")
    data = load_jsonl(file_path)
    print(f"Loaded {len(data)} records from {jsonl_file}")

    for line in data:
        row = process_line(line)
        rows.append(row)

print(f"Processed {len(rows)} rows from all files.")
df = pd.DataFrame(rows)
df = df.sort_values(by='order')
print(f"Total rows: {len(df)}")
# save as excel file
excel_path = os.path.join(results_path, f"{object_name}_results.xlsx")
df.to_excel(excel_path, index=False)



Loading results\driller\results_driller_analysis_batch_1.jsonl
Loaded 6033 records from results_driller_analysis_batch_1.jsonl
Loading results\driller\results_driller_analysis_batch_2.jsonl
Loaded 6042 records from results_driller_analysis_batch_2.jsonl
Loading results\driller\results_driller_analysis_batch_3.jsonl
Loaded 6032 records from results_driller_analysis_batch_3.jsonl
Loading results\driller\results_driller_analysis_batch_4.jsonl
Loaded 1519 records from results_driller_analysis_batch_4.jsonl
Processed 19626 rows from all files.
Total rows: 19626


In [ ]:
# ### USADO APENAS PARA ATUALIZAR CUSTOM_ID E FILE_PATH NO DATAFRAME DO DUCK PORQUE ESTAVA EM UMA VERSÃO ANTIGA

# data = load_jsonl('duck_analysis_batch_safe.jsonl')
# for line in data:
#     custom_id = line['custom_id']
#     file_path = split_custom_id(custom_id)
#     order = custom_id.split('_')[1]
    
#     if order in df['order'].values:
#         df.loc[df['order'] == order, 'custom_id'] = custom_id
#         df.loc[df['order'] == order, 'file_path'] = file_path

#     else:
#         print(f"Order {order} not found in DataFrame.")

# df.to_excel(excel_path, index=False)

In [5]:
# go through components find unique components and create boolean columns for each component
unique_components = set()
for components in df['components']:
    for component in components.split(', '):
        unique_components.add(component.strip())
# False = 0 and True = 1
for component in unique_components:
    df[component] = df['components'].apply(lambda x: 1 if component in x else 0)
# save the updated DataFrame to excel
df.to_excel(excel_path, index=False)